# CBOW와 Skip-gram 2가지 방식의 Word2Vec 모델을 
# 케라스로 직접구현하여 
# 엘리스의 이상한나라의 이야기 e-북 단어유사도 행렬 만들기

<center>written by JihoKwak</center> 

## 라이브러리 임포팅

In [42]:
import numpy as np                                                             #행렬연산
from tensorflow.keras.models import Sequential                                 #계층구조 모델 객체
from tensorflow.keras.layers import Dense, Embedding, GlobalAvgPool1D, Flatten #케라스 모델의 레이어 객체
from tensorflow.keras.utils import to_categorical                              #원핫인코딩
from tensorflow.keras.utils import get_file                                    # 웹주소에 해당하는 파일을 다운로드하여 메모리에 저장
from tensorflow.keras.preprocessing.sequence import pad_sequences              #각 문장의 시퀀스 데이터 길이를 맞추는 패딩을 하기 위해 사용
from tensorflow.keras.preprocessing.text import Tokenizer                      #문장을 토큰으로 나누기 위해 사용
import gensim                                                                  # Word2Vec 객체 생성

## 코퍼스(문자열타입 문장들의 리스트) 준비

In [2]:
path = get_file('alice.txt', origin='http://www.gutenberg.org/files/11/11-0.txt') #동화 엘리스 이북을 다운로드

corpus = open(path, encoding='utf-8').readlines()[:300] #메모리에 있는 텍스트 객체를 읽어 리스트형 코퍼스로 변환

corpus = [sentence for sentence in corpus if sentence.count(' ') >= 2] # 두단어 이상의 문장만 추출

## 코퍼스를 정수시퀀스 데이터로 변환

In [3]:
tokenizer = Tokenizer() #토크나이저 생성
tokenizer.fit_on_texts(corpus) # 토크나이저에 코퍼스 적합
sequences = tokenizer.texts_to_sequences(corpus) # 코퍼스(문자열들의 리스트) -> 시퀀스(정수 리스트들의 리스트)

## 주요 파라미터 정의

In [4]:
ONE_HOT_DIM = len(tokenizer.word_index) + 1 #원핫인코딩의 차원(= 전체 단어종류의 수 + 1, 언노운단어 1개포함) 
EMBED_DIM = 100 #임베딩 출력 차원 = 히든레이어의 유닛수, 여기서는 100차원으로 설정함.
WINDOW_SIZE = 2 #CBOW 윈도우 크기. 중심단어의 왼쪽 2개 단어, 오른쪽 2개 단어를 묶어줄때 사용

## 시퀀스 데이터 디코딩하여 코퍼스 내용 확인

In [6]:
#인코딩된 시퀀스데이터 확인
sequences[1:4]

[[22, 117, 66, 17, 1, 67, 8, 350, 351, 29, 38, 352, 4, 27],
 [146, 38, 353, 354, 16, 355, 356, 5, 198, 5, 147, 32],
 [357, 67, 5, 118, 1, 358, 8, 1, 199, 148, 359, 360]]

In [7]:
#코퍼스로 디코딩
for i in range(3) :
    print(" ".join([tokenizer.index_word[idx] for idx in sequences[i]]))

﻿project gutenberg’s alice’s adventures in wonderland by lewis carroll
this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever you may copy it give it away or


## ★중요★ CBOW모델용 입력데이터/레이블데이터 생성

In [29]:
contexts = []                        #입력데이터가 담길 빈리스트 (리스트타입(주변단어들)의 정보가 추가될 예정)
labels   = []                        #레이블데이터가 담길 빈리스트 (정수타입(중심단어)의 정보가 추가될 예정)
for sequence in sequences:                      # 시퀀스리스트(문장집합)에서 개별시퀀스(문장)에 하나씩 반복적으로 접근
    word_length = len(sequence)                 # 시퀀스길이를 변수저장
    for index, word in enumerate(sequence):     # 시퀀스(문장)에 enumerate을 씌워서 인덱스(순서)와 중심단어를 각각 반복적으로 접근
        start = index - WINDOW_SIZE             # 중심단어 인덱스에서 뒤로 2칸 이동한 지점을 윈도우의 시작 인덱스로 지정(주변단어의 시작)
        end = index + WINDOW_SIZE               # 중심단어 인덱스에서 뒤로 2칸 이동한 지점을 윈도우의 마지막 인덱스로 지정(주변단어의 마지막)
        contexts.append(                        #윈도우 안의(start~end)의 단어들을 돌면서 인덱스가 0보다 크면서 시퀀스길이를 초과하지 않고, 중심단어를 제외한
            [sequence[i] for i in range(start, end + 1) if 0 <= i < word_length and i != index] #주변단어들을 리스트로 생성하여 contexts
        ) 
        labels.append(word)                             #중심단어는 레이블 데이터리스트에 추가
X_train = pad_sequences(contexts, maxlen=WINDOW_SIZE*2) #윈도우별로 들쭉날쭉한 주변단어길이를 최대 윈도우사이즈의 2배(즉, 왼쪽오른쪽)만큼 0으로 채워줌
y_train = to_categorical(labels, ONE_HOT_DIM)           # 중심단어 데이터는 단어종류의수+1 만큼의 차원으로 원핫인코딩

In [49]:
print(X_train[:3])
print(X_train.shape)
print(y_train[:3])
print(y_train.shape)

[[  0   0 349  65]
 [  0 348  65 115]
 [348 349 115  10]]
(2787, 4)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(2787, 765)


## ★중요★ 케라스 CBOW 모델링

In [41]:
cbow = Sequential() #계층구조 모델객체 생성
cbow.add(Embedding(input_dim=ONE_HOT_DIM, output_dim=EMBED_DIM, input_length=WINDOW_SIZE*2)) #임베딩 레이어 생성
#Embedding 레이어가 Dense레이어와 다른점은, 가중치W만 있고, 차원이 높은 시퀀스 데이터 처리에 Dense레이어보다 효율적인면이 있음.
#Dense레이어는 정수시퀀스가 아닌 원핫인코딩벡터를 입력받지만, Embedding 레이어는 정수시퀀스를 바로 입력받는 차이점이 있음.
cbow.add(GlobalAvgPool1D()) #주변단어 4개에 해당하는 두번째 차원을 평균으로 압축시킴.
cbow.add(Dense(ONE_HOT_DIM, activation='softmax')) #100차원을 입력으로 받고, 765(단어종류수+1)차원으로 출력하는 Fully-Connected Layer
cbow.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['acc']) 
#레이블의 2개 이상이므로, softmax-crossentropy 손실함수를 사용하는 'categorical_crossentropy'옵션을 준다.
cbow.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 4, 100)            76500     
_________________________________________________________________
global_average_pooling1d (Gl (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 765)               77265     
Total params: 153,765
Trainable params: 153,765
Non-trainable params: 0
_________________________________________________________________


## CBOW 모델훈련

In [43]:
history = cbow.fit(X_train, y_train, batch_size=64, epochs=20000) #모델훈련(시간이 오래걸리니 약 1000번정도만 진)

Epoch 1/20000
2787/2787 [==============================] - 0s 66us/sample - loss: 2.0289 - acc: 0.7976
Epoch 2/20000
2787/2787 [==============================] - 0s 58us/sample - loss: 1.9830 - acc: 0.7983
Epoch 3/20000
2787/2787 [==============================] - 0s 62us/sample - loss: 1.9457 - acc: 0.7987
Epoch 4/20000
2787/2787 [==============================] - 0s 58us/sample - loss: 1.9128 - acc: 0.7998
Epoch 5/20000
2787/2787 [==============================] - 0s 61us/sample - loss: 1.8830 - acc: 0.7991
Epoch 6/20000
2787/2787 [==============================] - 0s 59us/sample - loss: 1.8551 - acc: 0.8009
Epoch 7/20000
2787/2787 [==============================] - 0s 58us/sample - loss: 1.8283 - acc: 0.8012
Epoch 8/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 1.8032 - acc: 0.8019
Epoch 9/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 1.7792 - acc: 0.8023
Epoch 10/20000
2787/2787 [==============================] - 0s 61us/sampl

2787/2787 [==============================] - 0s 60us/sample - loss: 0.5354 - acc: 0.9358
Epoch 157/20000
2787/2787 [==============================] - 0s 58us/sample - loss: 0.5320 - acc: 0.9365
Epoch 158/20000
2787/2787 [==============================] - 0s 58us/sample - loss: 0.5286 - acc: 0.9361
Epoch 159/20000
2787/2787 [==============================] - 0s 57us/sample - loss: 0.5254 - acc: 0.9372
Epoch 160/20000
2787/2787 [==============================] - 0s 64us/sample - loss: 0.5222 - acc: 0.9372
Epoch 161/20000
2787/2787 [==============================] - 0s 56us/sample - loss: 0.5189 - acc: 0.9376
Epoch 162/20000
2787/2787 [==============================] - 0s 58us/sample - loss: 0.5158 - acc: 0.9390
Epoch 163/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.5128 - acc: 0.9383
Epoch 164/20000
2787/2787 [==============================] - 0s 59us/sample - loss: 0.5097 - acc: 0.9404
Epoch 165/20000
2787/2787 [==============================] - 0s 58us/sa

2787/2787 [==============================] - 0s 59us/sample - loss: 0.2754 - acc: 0.9699
Epoch 311/20000
2787/2787 [==============================] - 0s 58us/sample - loss: 0.2746 - acc: 0.9706
Epoch 312/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.2741 - acc: 0.9706
Epoch 313/20000
2787/2787 [==============================] - 0s 58us/sample - loss: 0.2733 - acc: 0.9717
Epoch 314/20000
2787/2787 [==============================] - 0s 61us/sample - loss: 0.2724 - acc: 0.9720
Epoch 315/20000
2787/2787 [==============================] - 0s 58us/sample - loss: 0.2719 - acc: 0.9706
Epoch 316/20000
2787/2787 [==============================] - 0s 58us/sample - loss: 0.2712 - acc: 0.9724
Epoch 317/20000
2787/2787 [==============================] - 0s 64us/sample - loss: 0.2704 - acc: 0.9724
Epoch 318/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.2697 - acc: 0.9717
Epoch 319/20000
2787/2787 [==============================] - 0s 57us/sa

2787/2787 [==============================] - 0s 59us/sample - loss: 0.2082 - acc: 0.9734
Epoch 465/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.2080 - acc: 0.9745
Epoch 466/20000
2787/2787 [==============================] - 0s 59us/sample - loss: 0.2076 - acc: 0.9717
Epoch 467/20000
2787/2787 [==============================] - 0s 59us/sample - loss: 0.2076 - acc: 0.9731
Epoch 468/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.2072 - acc: 0.9749
Epoch 469/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.2070 - acc: 0.9727
Epoch 470/20000
2787/2787 [==============================] - 0s 59us/sample - loss: 0.2068 - acc: 0.97310s - loss: 0.2059 - acc: 0.975
Epoch 471/20000
2787/2787 [==============================] - 0s 59us/sample - loss: 0.2065 - acc: 0.9720
Epoch 472/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.2064 - acc: 0.9731
Epoch 473/20000
2787/2787 [==============

Epoch 541/20000
2787/2787 [==============================] - 0s 59us/sample - loss: 0.1930 - acc: 0.9720
Epoch 542/20000
2787/2787 [==============================] - 0s 66us/sample - loss: 0.1929 - acc: 0.9720
Epoch 543/20000
2787/2787 [==============================] - 0s 71us/sample - loss: 0.1926 - acc: 0.9731
Epoch 544/20000
2787/2787 [==============================] - 0s 66us/sample - loss: 0.1924 - acc: 0.9738
Epoch 545/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1923 - acc: 0.9724
Epoch 546/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1921 - acc: 0.9713
Epoch 547/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1919 - acc: 0.9727
Epoch 548/20000
2787/2787 [==============================] - 0s 63us/sample - loss: 0.1919 - acc: 0.9731
Epoch 549/20000
2787/2787 [==============================] - 0s 62us/sample - loss: 0.1917 - acc: 0.9720
Epoch 550/20000
2787/2787 [============================

...

2787/2787 [==============================] - 0s 65us/sample - loss: 0.1487 - acc: 0.9702
Epoch 19319/20000
2787/2787 [==============================] - 0s 62us/sample - loss: 0.1487 - acc: 0.9695
Epoch 19320/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1488 - acc: 0.9717
Epoch 19321/20000
2787/2787 [==============================] - 0s 64us/sample - loss: 0.1488 - acc: 0.9717
Epoch 19322/20000
2787/2787 [==============================] - 0s 58us/sample - loss: 0.1487 - acc: 0.97130s - loss: 0.1536 - acc: 0.974
Epoch 19323/20000
2787/2787 [==============================] - 0s 59us/sample - loss: 0.1487 - acc: 0.9720
Epoch 19324/20000
2787/2787 [==============================] - 0s 61us/sample - loss: 0.1488 - acc: 0.9695
Epoch 19325/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1487 - acc: 0.9713
Epoch 19326/20000
2787/2787 [==============================] - 0s 59us/sample - loss: 0.1486 - acc: 0.9706
Epoch 19327/20000
2787/27

2787/2787 [==============================] - 0s 58us/sample - loss: 0.1487 - acc: 0.9713
Epoch 19471/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1486 - acc: 0.9717
Epoch 19472/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1487 - acc: 0.9709
Epoch 19473/20000
2787/2787 [==============================] - 0s 63us/sample - loss: 0.1488 - acc: 0.9724
Epoch 19474/20000
2787/2787 [==============================] - 0s 62us/sample - loss: 0.1487 - acc: 0.9702
Epoch 19475/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1486 - acc: 0.9717
Epoch 19476/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1487 - acc: 0.9702
Epoch 19477/20000
2787/2787 [==============================] - 0s 58us/sample - loss: 0.1485 - acc: 0.9706
Epoch 19478/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1487 - acc: 0.9688
Epoch 19479/20000
2787/2787 [==========================

2787/2787 [==============================] - 0s 58us/sample - loss: 0.1487 - acc: 0.9724
Epoch 19621/20000
2787/2787 [==============================] - 0s 61us/sample - loss: 0.1487 - acc: 0.9720
Epoch 19622/20000
2787/2787 [==============================] - 0s 59us/sample - loss: 0.1488 - acc: 0.9709
Epoch 19623/20000
2787/2787 [==============================] - 0s 58us/sample - loss: 0.1489 - acc: 0.9695
Epoch 19624/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1488 - acc: 0.9695
Epoch 19625/20000
2787/2787 [==============================] - 0s 61us/sample - loss: 0.1487 - acc: 0.9724
Epoch 19626/20000
2787/2787 [==============================] - 0s 59us/sample - loss: 0.1487 - acc: 0.9731
Epoch 19627/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1487 - acc: 0.9706
Epoch 19628/20000
2787/2787 [==============================] - 0s 64us/sample - loss: 0.1488 - acc: 0.9720
Epoch 19629/20000
2787/2787 [==========================

2787/2787 [==============================] - 0s 60us/sample - loss: 0.1487 - acc: 0.97090s - loss: 0.1498 - acc: 0.970
Epoch 19772/20000
2787/2787 [==============================] - 0s 63us/sample - loss: 0.1486 - acc: 0.9717
Epoch 19773/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1487 - acc: 0.9699
Epoch 19774/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1487 - acc: 0.9717
Epoch 19775/20000
2787/2787 [==============================] - 0s 61us/sample - loss: 0.1485 - acc: 0.9720
Epoch 19776/20000
2787/2787 [==============================] - 0s 65us/sample - loss: 0.1487 - acc: 0.9709
Epoch 19777/20000
2787/2787 [==============================] - 0s 59us/sample - loss: 0.1486 - acc: 0.9695
Epoch 19778/20000
2787/2787 [==============================] - 0s 63us/sample - loss: 0.1487 - acc: 0.9727
Epoch 19779/20000
2787/2787 [==============================] - 0s 59us/sample - loss: 0.1487 - acc: 0.9709
Epoch 19780/20000
2787/27

2787/2787 [==============================] - 0s 59us/sample - loss: 0.1487 - acc: 0.9724
Epoch 19924/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1486 - acc: 0.9713
Epoch 19925/20000
2787/2787 [==============================] - 0s 61us/sample - loss: 0.1485 - acc: 0.9699
Epoch 19926/20000
2787/2787 [==============================] - 0s 58us/sample - loss: 0.1488 - acc: 0.9724
Epoch 19927/20000
2787/2787 [==============================] - 0s 59us/sample - loss: 0.1488 - acc: 0.9702
Epoch 19928/20000
2787/2787 [==============================] - 0s 60us/sample - loss: 0.1487 - acc: 0.9717
Epoch 19929/20000
2787/2787 [==============================] - 0s 64us/sample - loss: 0.1486 - acc: 0.9720
Epoch 19930/20000
2787/2787 [==============================] - ETA: 0s - loss: 0.1506 - acc: 0.970 - 0s 59us/sample - loss: 0.1487 - acc: 0.9713
Epoch 19931/20000
2787/2787 [==============================] - 0s 61us/sample - loss: 0.1488 - acc: 0.9706
Epoch 19932/20000

## ★중요★ Skip-gram모델용 입력데이터/레이블데이터 생성

In [31]:
couples = []
for seq in sequences : #문장별 시퀀스 하나씩 조회
    for idx, center in enumerate(seq) : # 중심단어의 시퀀스리스트상의 인덱스와 시퀀스정수값 순차 조회
        neighbors = seq[max(0, idx-WINDOW_SIZE): min(len(seq), idx+WINDOW_SIZE) + 1] #중심단어 포함 2+1+2 = 5 크기의 윈도우리스트 추출
        for neighbor in neighbors : 
            if neighbor is not center : # 윈도우리스트의 5개 값 중에서 중심단어만 제외하고 나머지 4개만 처리
                couples.append([center, neighbor]) # 중심단어, 이웃단어 쌍을 리스트에 저장

couples = np.vstack(couples) # 2차원 리스트를 넘파이 2차원 넘파이 행렬로 변환

In [47]:
X_train = couples[:,0] #중심단어 정수시퀀스는 그대로 입력데이터변수에 저장(원핫인코딩하지 않음)
y_train = to_categorical(couples[:,1], ONE_HOT_DIM) #이웃단어 정수시퀀스는 원핫인코딩을하여 (단어종류수+1)차원의 희소행렬을 레이블변수에 저장

In [48]:
print(X_train.shape)
print(y_train.shape)

(9780,)

## ★중요★ 케라스 Skip-gram모델링

In [52]:
skipgram = Sequential()                                                              #계층구조 모델객체 생성
skipgram.add(Embedding(input_dim=ONE_HOT_DIM, output_dim=EMBED_DIM, input_length=1)) #임베딩 레이어 생성
#Embedding 레이어에 대한 설명은 위와 같고, 
#input_length가 4가 아닌 1을 입력해준다. 왜냐하면, skip-gram은 중심단어 하나로 주변을 예측하는 것이기 때문(CBOW는 윈도우사이즈x2)
skipgram.add(Flatten()) #Embedding 레이어는 배치(문장수) x 시퀀스(단어종류수+1) x 히든유닛(임베딩차원) 의 3D텐서를 출력. 이를 2D로 펼침
skipgram.add(Dense(ONE_HOT_DIM, activation='softmax')) #100차원을 입력으로 받고, 765(단어종류수+1)차원으로 출력하는 Fully-Connected Layer
skipgram.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['acc']) 
#레이블의 2개 이상이므로, softmax-crossentropy 손실함수를 사용하는 'categorical_crossentropy'옵션을 준다.
skipgram.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 1, 100)            76500     
_________________________________________________________________
flatten_4 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 765)               77265     
Total params: 153,765
Trainable params: 153,765
Non-trainable params: 0
_________________________________________________________________


## Skip-gram 모델훈련

In [51]:
history = skipgram.fit(X_train, y_train, batch_size=64, epochs=3000) #모델훈련

Instructions for updating:
Use tf.cast instead.
Epoch 1/3000
9780/9780 [==============================] - 1s 101us/sample - loss: 6.5955 - acc: 0.0319
Epoch 2/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 6.5098 - acc: 0.0365
Epoch 3/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 6.4108 - acc: 0.0389
Epoch 4/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 6.2743 - acc: 0.0509
Epoch 5/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 6.1301 - acc: 0.0586
Epoch 6/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 6.0175 - acc: 0.0635
Epoch 7/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 5.9248 - acc: 0.0683
Epoch 8/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 5.8456 - acc: 0.0718
Epoch 9/3000
9780/9780 [==============================] - 1s 53us/sample - loss: 5.7775 - acc: 0.0752
Epoch 10/3000
9780/9780 [========

9780/9780 [==============================] - 1s 54us/sample - loss: 4.2275 - acc: 0.1345
Epoch 78/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 4.2143 - acc: 0.1359
Epoch 79/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 4.2010 - acc: 0.13590s - loss: 4.2018 - ac
Epoch 80/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 4.1880 - acc: 0.1346
Epoch 81/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 4.1753 - acc: 0.1358
Epoch 82/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 4.1626 - acc: 0.1353
Epoch 83/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 4.1501 - acc: 0.1348
Epoch 84/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 4.1380 - acc: 0.1346
Epoch 85/3000
9780/9780 [==============================] - 1s 58us/sample - loss: 4.1259 - acc: 0.1363
Epoch 86/3000
9780/9780 [==============================] - 1s 55u

9780/9780 [==============================] - 1s 56us/sample - loss: 3.6207 - acc: 0.1308
Epoch 156/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.6175 - acc: 0.1315
Epoch 157/3000
9780/9780 [==============================] - 1s 53us/sample - loss: 3.6133 - acc: 0.1321
Epoch 158/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 3.6101 - acc: 0.1305
Epoch 159/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.6068 - acc: 0.1300
Epoch 160/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 3.6034 - acc: 0.1306
Epoch 161/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.6001 - acc: 0.1308
Epoch 162/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.5969 - acc: 0.1302
Epoch 163/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.5936 - acc: 0.1304
Epoch 164/3000
9780/9780 [==============================] - 1s 53us/sample - lo

9780/9780 [==============================] - 1s 55us/sample - loss: 3.4614 - acc: 0.1260
Epoch 234/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4607 - acc: 0.1255
Epoch 235/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4599 - acc: 0.1253
Epoch 236/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4589 - acc: 0.1251
Epoch 237/3000
9780/9780 [==============================] - 1s 53us/sample - loss: 3.4580 - acc: 0.1233
Epoch 238/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4569 - acc: 0.1240
Epoch 239/3000
9780/9780 [==============================] - 1s 56us/sample - loss: 3.4560 - acc: 0.1253
Epoch 240/3000
9780/9780 [==============================] - 1s 59us/sample - loss: 3.4549 - acc: 0.1258
Epoch 241/3000
9780/9780 [==============================] - 1s 56us/sample - loss: 3.4539 - acc: 0.1262
Epoch 242/3000
9780/9780 [==============================] - 1s 54us/sample - lo

9780/9780 [==============================] - 1s 53us/sample - loss: 3.4167 - acc: 0.1215
Epoch 312/3000
9780/9780 [==============================] - 1s 57us/sample - loss: 3.4161 - acc: 0.1202
Epoch 313/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 3.4161 - acc: 0.1219
Epoch 314/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4150 - acc: 0.1206
Epoch 315/3000
9780/9780 [==============================] - 1s 53us/sample - loss: 3.4151 - acc: 0.1211
Epoch 316/3000
9780/9780 [==============================] - 1s 53us/sample - loss: 3.4152 - acc: 0.1212
Epoch 317/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4145 - acc: 0.1203
Epoch 318/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4143 - acc: 0.1198
Epoch 319/3000
9780/9780 [==============================] - 1s 53us/sample - loss: 3.4143 - acc: 0.1208
Epoch 320/3000
9780/9780 [==============================] - 1s 54us/sample - lo

...

9780/9780 [==============================] - 1s 54us/sample - loss: 3.4474 - acc: 0.1125
Epoch 2639/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4475 - acc: 0.1125
Epoch 2640/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4482 - acc: 0.1127
Epoch 2641/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4480 - acc: 0.1115
Epoch 2642/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4481 - acc: 0.1097
Epoch 2643/3000
9780/9780 [==============================] - 1s 53us/sample - loss: 3.4471 - acc: 0.1109
Epoch 2644/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4477 - acc: 0.1120
Epoch 2645/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4475 - acc: 0.1107
Epoch 2646/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4480 - acc: 0.1099
Epoch 2647/3000
9780/9780 [==============================] - 1s 54us/sa

Epoch 2716/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 3.4496 - acc: 0.11210s - loss: 3.3896 - a
Epoch 2717/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4496 - acc: 0.1119
Epoch 2718/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4496 - acc: 0.1073
Epoch 2719/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4499 - acc: 0.1125
Epoch 2720/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4492 - acc: 0.1082
Epoch 2721/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4503 - acc: 0.1130
Epoch 2722/3000
9780/9780 [==============================] - 1s 58us/sample - loss: 3.4499 - acc: 0.1099
Epoch 2723/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 3.4502 - acc: 0.1094
Epoch 2724/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4494 - acc: 0.1145
Epoch 2725/3000
9780/9780 [=======

9780/9780 [==============================] - 1s 55us/sample - loss: 3.4513 - acc: 0.1100
Epoch 2794/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 3.4510 - acc: 0.1136
Epoch 2795/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4510 - acc: 0.1109
Epoch 2796/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4512 - acc: 0.11070s - loss: 3.3874 - ac
Epoch 2797/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 3.4527 - acc: 0.1115
Epoch 2798/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 3.4515 - acc: 0.1120
Epoch 2799/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4524 - acc: 0.1108
Epoch 2800/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4519 - acc: 0.1111
Epoch 2801/3000
9780/9780 [==============================] - 1s 56us/sample - loss: 3.4524 - acc: 0.1086
Epoch 2802/3000
9780/9780 [======================

9780/9780 [==============================] - 1s 54us/sample - loss: 3.4532 - acc: 0.1121
Epoch 2871/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4534 - acc: 0.1106
Epoch 2872/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4537 - acc: 0.1128
Epoch 2873/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 3.4537 - acc: 0.1116
Epoch 2874/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 3.4537 - acc: 0.1100
Epoch 2875/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4533 - acc: 0.1092
Epoch 2876/3000
9780/9780 [==============================] - 1s 57us/sample - loss: 3.4529 - acc: 0.1126
Epoch 2877/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4540 - acc: 0.1122
Epoch 2878/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4534 - acc: 0.1124
Epoch 2879/3000
9780/9780 [==============================] - 1s 54us/sa

9780/9780 [==============================] - 1s 54us/sample - loss: 3.4552 - acc: 0.1136
Epoch 2948/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 3.4560 - acc: 0.1117
Epoch 2949/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4558 - acc: 0.1115
Epoch 2950/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4559 - acc: 0.1094
Epoch 2951/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 3.4550 - acc: 0.1084
Epoch 2952/3000
9780/9780 [==============================] - 1s 56us/sample - loss: 3.4555 - acc: 0.1076
Epoch 2953/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4552 - acc: 0.1105
Epoch 2954/3000
9780/9780 [==============================] - 1s 54us/sample - loss: 3.4541 - acc: 0.1154
Epoch 2955/3000
9780/9780 [==============================] - 1s 55us/sample - loss: 3.4555 - acc: 0.1129
Epoch 2956/3000
9780/9780 [==============================] - 1s 55us/sa

## gensim 라이브러리를 이용해, 임베딩벡터를 Word2Vec 객체로 생성

In [ ]:
import os
os.makedirs("./data/", exist_ok=True) # 파일을 생성할 디렉토리를 생성

### 1) CBOW 임베딩벡터 처리 및 Word2Vec 객체생성

In [64]:
with open("./data/alice_cbow_100d_vector.txt", "w", encoding='utf-8') as f: # with 컨텍스트 관리자를 사용하면 f.close() 마지막에 사용하지 않아도됨.
    f.write('{} {}\n'.format(ONE_HOT_DIM-1, EMBED_DIM))                     #맨 윗줄에 단어종류수와 임베딩 차원을 기록
    embedding_matrix = cbow.get_weights()[0]                                #훈련된 모델에서 가중치 벡터를 추출하여 변수저장
    for word, i in tokenizer.word_index.items() :                           #단어-인덱스 사전을 하나씩 조회
        str_vec = ' '.join(map(str, list(embedding_matrix[i, :])))          # 임베딩행렬에서 인덱스에 맞는 개별벡터를 공백구분자로 문자열변환
        f.write('{} {}\n'.format(word, str_vec))                            # 단어와 임베딩벡터를 한줄씩 기록

In [65]:
cbow_w2v = gensim.models.KeyedVectors.load_word2vec_format("./data/alice_cbow_100d_vector.txt", binary=False) #임베딩데이터를 객체로 로드

In [67]:
cbow_w2v.most_similar(positive=['alice']) #alice 단어와 유사도가 높은 단어를 내림차순으로 출력

[('anxiously', 0.5236098766326904),
 ('dropped', 0.4307931959629059),
 ('violently', 0.4196436405181885),
 ('severely', 0.4073325991630554),
 ('gave', 0.377197802066803),
 ('lately', 0.36055031418800354),
 ('playing', 0.3441125750541687),
 ('managed', 0.3172294795513153),
 ('presently', 0.315362811088562),
 ('planning', 0.30474627017974854)]

### 2) Skip-gram 임베딩벡터 처리 및 Word2Vec 객체생성

In [53]:
with open("./data/alice_skipgram_100d_vector.txt", "w", encoding='utf-8') as f: #with 컨텍스트 관리자를 사용하면 f.close() 마지막에 사용하지 않아도됨.
    f.write('{} {}\n'.format(ONE_HOT_DIM-1, EMBED_DIM)) #맨 윗줄에 단어종류수와 임베딩 차원을 기록
    embedding_matrix = skipgram.get_weights()[0] #훈련된 모델에서 가중치 벡터를 추출하여 변수저장
    for word, i in tokenizer.word_index.items() : #단어-인덱스 사전을 하나씩 조회
        str_vec = ' '.join(map(str, list(embedding_matrix[i, :]))) # 임베딩행렬에서 인덱스에 맞는 개별벡터를 공백구분자로 문자열변환
        f.write('{} {}\n'.format(word, str_vec)) # 단어와 임베딩벡터를 한줄씩 기록

In [54]:
skipgram_w2v = gensim.models.KeyedVectors.load_word2vec_format("./data/alice_skipgram_100d_vector.txt", binary=False) #임베딩데이터를 객체로 로드

In [55]:
skipgram_w2v.most_similar(positive=['alice']) #alice 단어와 유사도가 높은 단어를 내림차순으로 출력

[('‘oh', 0.28033241629600525),
 ('shan’t', 0.27819472551345825),
 ('perhaps', 0.27111393213272095),
 ('chain', 0.2662968933582306),
 ('through', 0.23889237642288208),
 ('won’t', 0.22940471768379211),
 ('fancy', 0.22921088337898254),
 ('lit', 0.2214980274438858),
 ('hope', 0.21725526452064514),
 ('felt', 0.21653945744037628)]

## 구글뉴스기사로 사전훈련된 Word2Vec 사용

### 다운로드 주소 : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
./data/ 디렉토리 아래 압축을 풀어 데이터 준비

In [ ]:
import numpy as np
filepath = "./data/GoogleNews-vectors-negative300.bin"

embeddings_index = {} # 단어사전용 빈딕셔너리 생성
wv_from_bin = gensim.KeyedVectors.load_word2vec_format(filepath, binary=True) #Word2vec 객체로 임베딩 데이터 로딩

In [58]:
wv_from_bin.most_similar(positive=['alice']) #alice 단어와 유사도가 높은 단어를 내림차순으로 출력

[('nikki', 0.6746107339859009),
 ('christina', 0.6611028909683228),
 ('joshua', 0.6600387692451477),
 ('joel', 0.6595911383628845),
 ('disney', 0.6571455597877502),
 ('rebecca', 0.648557186126709),
 ('danielle', 0.6450589895248413),
 ('fiona', 0.6436316967010498),
 ('julia', 0.6429926156997681),
 ('ernie', 0.6416751146316528)]

In [57]:
for word, vector in zip(wv_from_bin.vocab, wv_from_bin.vectors): # word2vec의 단어와 벡터 데이터를 순회하면서 단어사전 딕셔너리에 저장
    coefs = np.asarray(vector, dtype='float32')
    embeddings_index[word] = coefs